<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/Llama_3_8B_for_text_to_SQL_upload_model_hf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install peft --quiet
! pip install datasets trl ninja packaging --quiet

In [ ]:
# Install Pytorch & other libraries
!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet

#FlashAttention only supports Ampere GPUs or newer. #NEED A100 IN GOOGLE COLAB
#!pip install -U transformers
!pip install -U flash-attn --no-build-isolation --quiet


! pip install peft --quiet
! pip install datasets trl ninja packaging --quiet

# Uncomment only if you're using A100 GPU
#!pip install flash-attn --no-build-isolation
!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet

In [3]:
!pip install colab-env --quiet
!pip install huggingface_hub -q

import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

Mounted at /content/gdrive


In [4]:
#!pip install huggingface_hub --quiet
from huggingface_hub import HfApi

api = HfApi()
api.get_token_permission(token=access_token_write)
#api.set_access_token(access_token)


# frankmorales2020/Mistral-7B-text-to-sql Good

repo_id = "my-awesome-model-poc"
#api.create_repo(repo_id=repo_id, private=False)

#api.delete_repo(repo_id=repo_id)

#api.upload_folder(
#    folder_path="./model",
#    repo_id=repo_id,
#    repo_type="model",
#)


In [5]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
)

In [6]:
#frankmorales2020/Mistral-7B-text-to-sql
# Chose the base model you want
#model_name = "frankmorales2020/Mistral-7B-text-to-sql" # 01/03/2023
#frankmorales2020/Mistral-7B-text-to-sql-flash-attention-2
model_name = "frankmorales2020/Meta-Llama-3-8B-text-to-sql-flash-attention-2" # 04/03/2024
# set device
device = 'cuda'
#v Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
# We redefine the pad_token and pad_token_id with out of vocabulary token (unk_token)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

tokenizer_config.json:   0%|          | 0.00/51.2k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/419 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [8]:
!pip install huggingface_hub -q

In [9]:
from datasets import load_dataset
from huggingface_hub import hf_hub_url

In [10]:
from datasets import load_dataset

# Convert dataset to OAI messages
system_message = """You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.
SCHEMA:
{schema}"""

def create_conversation(sample):
  return {
    "messages": [
      {"role": "system", "content": system_message.format(schema=sample["context"])},
      {"role": "user", "content": sample["question"]},
      {"role": "assistant", "content": sample["answer"]}
    ]
  }

# Load dataset from the hub
# jtjt520j/CSpider_sql_create_context
# b-mc2/sql-create-context
#dataset = load_dataset("jtjt520j/CSpider_sql_create_context", split="train")
dataset = load_dataset("b-mc2/sql-create-context", split="train")
dataset = dataset.shuffle().select(range(12500))

# Convert dataset to OAI messages
dataset = dataset.map(create_conversation, remove_columns=dataset.features,batched=False)
# split dataset into 10,000 training samples and 2,500 test samples
dataset = dataset.train_test_split(test_size=2500/12500)

print(dataset["train"][345]["messages"])

# save datasets to disk
dataset["train"].to_json("train_dataset.json", orient="records")
dataset["test"].to_json("test_dataset.json", orient="records")

Generating train split:   0%|          | 0/78577 [00:00<?, ? examples/s]

Map:   0%|          | 0/12500 [00:00<?, ? examples/s]

[{'content': 'You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.\nSCHEMA:\nCREATE TABLE table_224616_1 (location_of_venue VARCHAR, venue VARCHAR)', 'role': 'system'}, {'content': 'Where is the Bellerive Country Club venue located?', 'role': 'user'}, {'content': 'SELECT location_of_venue FROM table_224616_1 WHERE venue = "Bellerive country Club"', 'role': 'assistant'}]


Creating json from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

1189283

In [11]:
from datasets import load_dataset
from random import randint

# Load our test dataset and Tokenizer again
#tokenizer = AutoTokenizer.from_pretrained("frankmorales2020/Mistral-7B-text-to-sql") # 01/03/2024
tokenizer = AutoTokenizer.from_pretrained("frankmorales2020/Mistral-7B-text-to-sql-flash-attention-2") # 04/03/2024
eval_dataset = load_dataset("json", data_files="test_dataset.json", split="train")
rand_idx = randint(0, len(eval_dataset))

tokenizer_config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Generating train split: 0 examples [00:00, ? examples/s]

In [12]:
# generate the same prompt as for the first local test
prompt = tokenizer.apply_chat_template(eval_dataset[rand_idx]["messages"][:2], tokenize=False, add_generation_prompt=True)
request= {"inputs":prompt,"parameters":{"temperature":0.2, "top_p": 0.95, "max_new_tokens": 256}}


Parameter-Efficient Fine-Tuning (PEFT)

In [14]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

#peft_model_id = "frankmorales2020/Mistral-7B-text-to-sql-flash-attention-2" # 04/03/2024 and 10/03/2024

peft_model_id = "frankmorales2020/Meta-Llama-3-8B-text-to-sql-flash-attention-2" #22/05/2024
# peft_model_id = args.output_dir

# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

adapter_config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


adapter_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForC

In [15]:
from datasets import load_dataset
from random import randint


# Load our test dataset
eval_dataset = load_dataset("json", data_files="test_dataset.json", split="train")
rand_idx = randint(0, len(eval_dataset))

# Test on sample
prompt = pipe.tokenizer.apply_chat_template(eval_dataset[rand_idx]["messages"][:2], tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=False, temperature=0.1, top_k=50, top_p=0.1, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [17]:
print(f"Query:\n{eval_dataset[rand_idx]['messages'][1]['content']}")
print(f"Original Answer:\n{eval_dataset[rand_idx]['messages'][2]['content']}")
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")

Query:
What percentage of browsers were using Safari during the period in which 31.27% were using Firefox?
Original Answer:
SELECT safari FROM table_name_62 WHERE firefox = "31.27%"
Generated Answer:
SELECT safari FROM table_name_62 WHERE firefox = "31.27%"


https://huggingface.co/frankmorales2020

In [20]:
from tqdm import tqdm

# Load our test dataset
eval_dataset = load_dataset("json", data_files="test_dataset.json", split="train")
rand_idx = randint(0, len(eval_dataset))

def evaluate(sample):
    prompt = pipe.tokenizer.apply_chat_template(sample["messages"][:2], tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)
    predicted_answer = outputs[0]['generated_text'][len(prompt):].strip()
    print()
    print()
    print('Question: %s\n'%sample["messages"][1]["content"])
    #print(sample["messages"][2]["content"])
    #print('Predicted Answer: %s'%sample["messages"][2]["content"])
    print()
    if predicted_answer == sample["messages"][2]["content"]:
        #print('Success!')
        print('Success Answer: %s'%sample["messages"][2]["content"])
        return 1
    else:
        print('Real Answer: %s'%predicted_answer)
        print('Failed Answer: %s'%sample["messages"][2]["content"])
        return 0

success_rate = []

number_of_eval_samples = 10
# iterate over eval dataset and predict
for s in tqdm(eval_dataset.shuffle().select(range(number_of_eval_samples))):
    success_rate.append(evaluate(s))

# compute accuracy
accuracy = sum(success_rate)/len(success_rate)

print(f"Accuracy: {accuracy*100:.2f}%")

 10%|█         | 1/10 [00:01<00:12,  1.34s/it]



Question: On what date were the Ottawa Swans the away team?


Success Answer: SELECT date FROM table_name_25 WHERE away = "ottawa swans"


 20%|██        | 2/10 [00:03<00:13,  1.64s/it]



Question: How many Airspeed Fleet Shadower aircraft are retired?


Success Answer: SELECT retired FROM table_13605170_2 WHERE aircraft_type = "Airspeed Fleet Shadower"


 30%|███       | 3/10 [00:04<00:10,  1.46s/it]

...



Question: Which Year has a Score of 5–7, 4–6, 6–3, 1–6?


Success Answer: SELECT year FROM table_name_88 WHERE score = "5–7, 4–6, 6–3, 1–6"


 90%|█████████ | 9/10 [00:16<00:01,  1.90s/it]



Question: What were the co-drivers in 2008?


Success Answer: SELECT co_drivers FROM table_name_76 WHERE year = 2008


100%|██████████| 10/10 [00:18<00:00,  1.83s/it]



Question: What was the maximum week that had attendance of 10738?


Success Answer: SELECT MAX(week) FROM table_24951872_2 WHERE attendance = 10738
Accuracy: 90.00%


When evaluated on 10 samples from the evaluation dataset, our model achieved an impressive accuracy of 90.00%.


In [19]:
from tqdm import tqdm

# Load our test dataset
eval_dataset = load_dataset("json", data_files="test_dataset.json", split="train")
rand_idx = randint(0, len(eval_dataset))

def evaluate(sample):
    prompt = pipe.tokenizer.apply_chat_template(sample["messages"][:2], tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)
    predicted_answer = outputs[0]['generated_text'][len(prompt):].strip()
    if predicted_answer == sample["messages"][2]["content"]:
        return 1
    else:
        return 0

success_rate = []
number_of_eval_samples = 1000
# iterate over eval dataset and predict
for s in tqdm(eval_dataset.shuffle().select(range(number_of_eval_samples))):
    success_rate.append(evaluate(s))

# compute accuracy
accuracy = sum(success_rate)/len(success_rate)

print(f"Accuracy: {accuracy*100:.2f}%")

100%|██████████| 1000/1000 [29:00<00:00,  1.74s/it]

Accuracy: 81.00%



When evaluated on 1000 samples from the evaluation dataset, our model achieved an impressive accuracy of 81.00%. However, there's room for improvement. We could enhance the model's performance by exploring techniques like few-shot learning, RAG, and Self-healing to generate the SQL query.